In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abduljaweed","key":"db93b78309e7cdc0772ce894bd6cc0ee"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle\.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [00:21<00:00, 179MB/s]
100% 2.70G/2.70G [00:21<00:00, 133MB/s]


In [5]:
!unzip -q new-plant-diseases-dataset.zip

# Import Libraries

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from time import perf_counter 
import os
import seaborn as sns
from PIL import Image 
from PIL import ImageEnhance
from skimage.io import imread
import matplotlib.pyplot as plt
import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, Conv2D 
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dense, Dropout
K.clear_session()

ValueError: The name 'tf.StaticHashTable' has already been registered to a serializable class. Found: <class 'tensorflow.python.ops.lookup_ops.StaticHashTable'>

# Define Batch Specifications

In [ ]:
## Defining batch specfications

# batch_size = 100
batch_size = 16
img_height = 160
img_width = 160

In [3]:
from tensorflow.keras import mixed_precision

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


# Load the Dataset

In [4]:
training_data = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\ideapad\Desktop\New folder\archive (1)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 70295 files belonging to 38 classes.


# Getting the Validation Data

In [5]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\ideapad\Desktop\New folder\archive (1)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 17572 files belonging to 38 classes.


# Storing the Label Names

In [6]:
target_names = training_data.class_names
print(target_names)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

# Building MobileNet Architecture

In [7]:
# Building MobileNet Architecture

base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dense(len(target_names), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_160 (Functio  (None, 5, 5, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 256)               6553856   
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1

# Compile the Model

In [8]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training and Validation

In [10]:
history = model.fit(training_data,validation_data= validation_data,epochs = 100)

Epoch 1/100
4394/4394 [==============================] - 599s 136ms/step - loss: 0.0370 - accuracy: 0.9878 - val_loss: 0.1060 - val_accuracy: 0.9697
Epoch 2/100
4394/4394 [==============================] - ETA: 0s - loss: 0.0333 - accuracy: 0.9891

KeyboardInterrupt: 

# Saving the Model

In [10]:
model.save("MobileNet.h5")

In [5]:
!pip install tensorflow_model_optimization

In [6]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

# Load your pre-trained model
model = tf.keras.models.load_model('MobileNet.h5')

# Create a new model with quantization annotations
quant_annotate_model = tf.keras.Sequential()

# Iterate over the layers and add quantization annotation
for layer in model.layers:
    annotated_layer = tfmot.quantization.keras.quantize_annotate_layer(layer)
    quant_annotate_model.add(annotated_layer)

# Apply quantization to the model
quantize_model = tfmot.quantization.keras.quantize_apply(quant_annotate_model)

# Compile and evaluate the quantized model
quantize_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# loss, accuracy = quantize_model.evaluate(x_test, y_test)
# print("Quantized model - Loss: {:.4f}, Accuracy: {:.4f}".format(loss, accuracy))

ValueError: `to_annotate` can only be a `tf.keras.layers.Layer` instance. You passed an instance of type: Functional.

In [ ]:
# Save the quantized model
quantize_model.save('quantized_model.h5')

In [7]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

# Load or define your Keras model
model = tf.keras.models.load_model('your_model.h5')

# Create a new model for quantization annotation
quant_annotate_model = tf.keras.Sequential()

# Iterate over the layers and add quantization annotation
for layer in model.layers:
    annotated_layer = tfmot.quantization.keras.quantize_annotate_layer(layer)
    quant_annotate_model.add(annotated_layer)

# Apply quantization to the model
quant_annotate_model = tfmot.quantization.keras.quantize_apply(quant_annotate_model)

# Save the quantized model
quant_annotate_model.save('quantized_model.h5')

OSError: No file or directory found at your_model.h5

In [15]:
import tensorflow_model_optimization as tfmot

# Create an empty list to store the annotated layers
annotated_layers = []

# Iterate over the layers of the model and annotate each layer separately
for layer in full_model.layers:
    annotated_layer = tfmot.quantization.keras.quantize_annotate_layer(layer)
    annotated_layers.append(annotated_layer)

# Create a new model with the annotated layers
quant_annotate_model = tf.keras.Sequential(annotated_layers)

# Apply quantization to the model
quantized_model = tfmot.quantization.keras.quantize_apply(quant_annotate_model)


AttributeError: 'ConcreteFunction' object has no attribute 'layers'

In [16]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

# Load or define your TensorFlow model
# Replace this with your own model loading or creation code
model = tf.keras.models.load_model('MobileNet.h5')

# Convert the model to a TensorFlow ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Apply quantization to the model
quantized_model = tfmot.quantization.keras.quantize_apply(full_model)

# Save the quantized model
# Replace 'path_to_save_quantized_model' with your desired save path
tf.keras.models.save_model(quantized_model, 'quantized_model.h5')


ValueError: `model` can only be a `tf.keras.Model` instance.You passed an instance of type: ConcreteFunction.